# Power Point Presentation maker

## Configs
Load the Api keys from the environment, and create some basic util functions

In [1]:
from __future__ import annotations

import os
import string
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    msg = "Missing API_KEY"
    raise OSError(msg)

# Print prompt results
print_bool = True


def remove_punctuation(text: str) -> str:
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

## Using OpenAI chat with structured Format to get the PPT content

[OpenAI docs - structured format](https://platform.openai.com/docs/guides/structured-outputs)
[Pydantic](https://docs.pydantic.dev/latest/) is being used to pass a Structured Format to OpenAI in order to receive correctly formatted response. 

In [2]:
from pydantic import BaseModel


class SingleSlide(BaseModel):
    Slide_Number: int
    Slide_Title: str
    Slide_Text: str
    Image_Prompt: str


class ResponseFormat(BaseModel):
    Slide: list[SingleSlide]


class ResponseFormatBase(BaseModel):
    Text: str


class Notes(BaseModel):
    Slide_Number: int
    Slide_Title: str
    Presenter_Notes: str
    Additional_info: str


class NotesFormat(BaseModel):
    Note: list[Notes]

In [3]:
from typing import TypeVar

from pydantic import ValidationError

T = TypeVar("T", bound=BaseModel)


def validate_response(response: T | None, cls: type[T]) -> T:
    try:
        return cls.model_validate(response)
    except ValidationError as e:
        print(f"Error Validating response -> {response} \n {e}")
        raise

In [4]:
from openai import OpenAI


def chat_completion(prompt: str, response_format: type[T]) -> T | None:
    client = OpenAI(api_key=api_key)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_format=response_format,
    )
    return completion.choices[0].message.parsed

## Prompts
2 Prompts are being used to generate the response, the first generates a PPT outline, and the second uses that outline to generate the PPT content.

## Variables
- **Topic**: Main topic of the presentation: str
- **Slides**: Number of slides: int
- **Audience**: Level of experience of the audience with a specific theme: str
- **Points**: Talking points to include in the presentation: list[list[str]]
- **Additional Info**: Additional Info to get passed to the prompt:list[str]

### Talking Points
Talking Points are in a list format: 
```
[
    [Talking point,Example 1.1,Example 1.2,...],
    [Talking point 2, Example 2,...],
    ...
]
```

### Additional Info
Additional information that can be passed in the prompt:
```
[Info1, Info2,...]
```

In [14]:
topic = "Practical Uses of AI on a Daily Basis for Developers"
slides = 10
audience = "Experienced"
points: list[list[str]] = [
    ["Search information", "Perplexity AI and Chat GPT Search"],
    ["Research and Study", "NotebookLM"],
    ["Note-Taking", "Notion AI", "Obsidian with AI plugins"],
    ["Automation and API usage", "API usage with programming language",
        "Should be put at the end of the presentation"],
    ["Meetings", "Otter AI"],
]
additional_info = [
    "Search for additional tools that fit the topic",
    "Move the Automation and API usage slide to the end of the presentation",
    ]

In [15]:
from prompts import PPT_prompt, generalized_outline_prompt, notes_prompt

outline_prompt = generalized_outline_prompt(
    slides, topic, audience, points, additional_info,
)
first_chat = validate_response(
    chat_completion(outline_prompt, ResponseFormatBase),
    ResponseFormatBase,
)
if print_bool:
    print(first_chat.Text)

# Outline for PowerPoint Presentation: Practical Uses of AI on a Daily Basis for Developers

## Slide 1: Introduction
- **Title:** Practical Uses of AI on a Daily Basis for Developers  
- **Key Points:**  
  - Presenter Name: [Insert Presenter Name]  
  - Overview of AI's impact on the development process.  
  - Brief introduction of topics to be covered.  

## Slide 2: Enhanced Information Retrieval
- **Title:** Smart Search Tools  
- **Key Points:**  
  - **Perplexity AI:** An AI-powered search engine that summarizes results and understands context.  
  - **ChatGPT:** Use for getting precise answers and code snippets quickly.  
  - **Benefits:** Increased efficiency in finding relevant information and documentation.  

## Slide 3: Research and Study Aids  
- **Title:** AI for Learning and Research  
- **Key Points:**  
  - **NotebookLM:** AI-driven notebook enhancing productivity with smart organization and search capabilities.  
  - **Code Interpreter Tools:** AI tools that help und

In [17]:
second_chat = validate_response(
    chat_completion(
        f"{PPT_prompt}\n```# Outline\n + {first_chat.Text}```", ResponseFormat,
    ),
    ResponseFormat,
)

second_chat_text: list[str] = []
second_chat_prompts: list[str] = []

for slide in second_chat.Slide:
    second_chat_text.append(
        f"{slide.Slide_Number}: {slide.Slide_Title}\n"
        f"{slide.Slide_Text}\n",
    )
    second_chat_prompts.append(f" {slide.Image_Prompt}\n")

if print_bool:
    for text, prompt in zip(second_chat_text, second_chat_prompts):
        print(f"{text}{prompt}\n")

1: Introduction
Practical Uses of AI on a Daily Basis for Developers
Presenter Name: [Insert Presenter Name]
Overview of AI's impact on the development process
Brief introduction of topics to be covered
 "An abstract depiction of artificial intelligence symbols merging with coding elements, creating a bridge between technology and software development."


2: Smart Search Tools
Perplexity AI: An AI-powered search engine that summarizes results and understands context.
ChatGPT: Use for getting precise answers and code snippets quickly.
Benefits: Increased efficiency in finding relevant information and documentation.
 "An abstract representation of a digital search interface with AI elements, showcasing a fluid interaction of data and information retrieval concepts."


3: AI for Learning and Research
NotebookLM: AI-driven notebook enhancing productivity with smart organization and search capabilities.
Code Interpreter Tools: AI tools that help understand code libraries and frameworks.
Ben

# Notes

Create a markdown file with notes to accompany the presentation

In [18]:
second_chat_string: str = "".join(second_chat_text)
notes_chat = validate_response(
    chat_completion(
        notes_prompt + f"# Slide Content\n{second_chat_string}",
        NotesFormat,
    ),
    NotesFormat,
)
notes_string: str = "".join(
    [
        f"# Slide{note.Slide_Number}: {note.Slide_Title}\n\n"
        f"## Presenter Notes\n{note.Presenter_Notes}\n\n"
        f"## Additional Information:\n{note.Additional_info}\n\n\n"
        for note in notes_chat.Note
    ],
)
if print_bool:
    print(notes_string)

# Slide1: Introduction

## Presenter Notes
Welcome, everyone! I’m [Insert Presenter Name], and today we’re diving into the practical uses of AI in development. In a landscape that’s rapidly evolving, AI is no longer just a buzzword; it’s a game-changer for developers like yourself. We’ll explore how AI impacts your daily workflow, boosts productivity, and enhances the overall coding experience. Our discussion will cover smart search tools, AI for learning and research, note-taking, meeting tools, debugging solutions, and much more. You’ll leave today with practical insights that you can implement right away!

## Additional Information:
AI is reshaping how we write code, manage projects, and collaborate with teams. Make sure to think about your own experiences with AI as we go through the presentation.


# Slide2: Smart Search Tools

## Presenter Notes
Let’s start with smart search tools. Perplexity AI and ChatGPT have changed the game in how we find information. For instance, Perplexit

In [19]:
with Path("results/notes.md").open("w", encoding="utf-8") as f:
    f.write(notes_string)

## Function to create images with Stable Diffusion

Using a local Stable Diffusion API to create the images, currently using the [WebUI Forge API](https://github.com/lllyasviel/stable-diffusion-webui-forge).

In [20]:
import base64

import requests

# Define the URL and the payload to send.
url = "http://127.0.0.1:7860"


def imagecreator(prompt: str, name: str, width: int, height: int) -> None:
    """Access local Stable Diffusion to create images.

    Args:
        prompt (str): prompt for image creation
        name (str): Name of the resulting image
        width (int): Image Width
        height (int): Image Height

    """
    payload = {
        "prompt": prompt,
        "styles": ["Avant-garde"],
        "steps": 20,
        "sampler_name": "DPM++ 2M",
        "scheduler": "Beta",
        "cfg_scale": 1,
        "width": width,
        "height": height,
    }

    # Send said payload to said URL through the API.
    response = requests.post(url=f"{url}/sdapi/v1/txt2img", json=payload, timeout=300)
    r = response.json()

    # Decode and save the image.
    with Path(f"images/{name}.png").open("wb") as f:
        f.write(base64.b64decode(r["images"][0]))

## Create a PPTX with the responses generated with ChatGPT and the Stable Diffusion images

Using the [Python-pptx](https://python-pptx.readthedocs.io/en/latest/index.html) library for PowerPoint creation

Check Available Layouts

In [21]:
from pptx import Presentation

prs = Presentation()
for idx, layout in enumerate(prs.slide_layouts):
    print(f"{idx}: {layout.name}")

0: Title Slide
1: Title and Content
2: Section Header
3: Two Content
4: Comparison
5: Title Only
6: Blank
7: Content with Caption
8: Picture with Caption
9: Title and Vertical Text
10: Vertical Title and Text


# PPT Creation
The Previous OpenAI chat returned a Title, Text and a prompt for image creation
This information is being added to basic empty slides

## Manual Step
After the PPT is created, the PPT is getting manually opened and the slides are organized using the designer option inside the software

In [22]:
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
title_slide_layout = prs.slide_layouts[0]

for ai_slide in second_chat.Slide:
    slide = prs.slides.add_slide(title_slide_layout)

    # Remove Punctuation from title
    slide_title = remove_punctuation(ai_slide.Slide_Title)

    # Add background image
    left = top = Inches(0)
    slide_width = prs.slide_width
    slide_height = prs.slide_height
    # Create Image
    imagecreator(
        ai_slide.Image_Prompt,
        slide_title,
        955,
        537,
    )
    # Add Image to slide
    pic = slide.shapes.add_picture(
        str(Path(f"images/{slide_title}.png")),
        left,
        top,
        slide_width,
        slide_height,
    )

    # Add Text
    title_shape = slide.shapes.title
    if title_shape is None:
        msg = "Shape is None"
        raise ValueError(msg)
    title_shape.text = ai_slide.Slide_Title

    body_shape = slide.shapes.placeholders[1]
    tf = body_shape.text_frame  # type: ignore[reportAttributeAccessIssue]
    tf.text = ai_slide.Slide_Text


prs.save(str(Path("results/test.pptx")))